In [1]:
import os
import os.path as op
import shutil
import glob
from nilearn import image
from nilearn import masking
import numpy as np
import scipy.stats
import pandas as pd
import datetime
import subprocess

In [ ]:
# Get tasks and cons from your BIDS/code/contrasts.tsv file
# Second column is either 'description' or 'desc' in constrasts.tsv
def get_cons_and_tasks(proj_dir, second_column='description'):
    df = pd.read_csv(proj_dir + "data/BIDS/code/contrasts_task-read.tsv", sep="\t")
    cons = []
    tasks = []
    for i, row in df.iterrows():
        cons.append("con_%i_%s" % (i+1, row[second_column].lower()))
        if row['task'] not in tasks:
            tasks.append(row['task'])
    return cons, tasks

In [ ]:
#Edit this line for your project
proj_dir = '/nese/mit/group/saxelab/projects/EMOfd/'

# COULD SET TO RUN ON SPECIFIC CONDS (or otherwise by list): 
# cons = [['con_1_tgn-gt-cgn', 'con_2_cgd-gt-cgn', 'con_3_cgf-gt-cgn',
#         'con_4_cgf-gt-cgd', 'con_5_cgd-gt-cgf', 'con_6_cgn', 'con_7_cgd', 
#         'con_8_cgf', 'con_9_tgn']] # this is an array of (arrays of cons per task)
# tasks = ['read']


# OR GET CONS AND TASKS PROGRAMMATICALLY:  
cons, tasks = get_cons_and_tasks(proj_dir)
cons = [cons] 



groups = ['ENDORSE', 'OPPOSE']

iterative = True #Did you do iterative subject-level modeling?

master_data = op.join(proj_dir, 'data/subject_lists/EMOfd_subject_info_211026.csv')


tier_proj_dir = proj_dir
tier_dir = op.join(tier_proj_dir,'TIER')
copes_dir = op.join(tier_dir,'original_data','copes')
varcopes_dir = op.join(tier_dir,'original_data','varcopes')
#masks_dir = op.join(tier_dir,'original_data','inverted_masks')
flame_outputdir_root = op.join(tier_dir,'analysis_data','group_flame')
do_allsubs_average = True

between_group_design_dir = op.join(tier_dir, 'analysis_data/group_flame/between_group/')
whole_group_design_dir = op.join(tier_dir, 'analysis_data/group_flame/whole_group/')

print(cons, tasks)

In [ ]:
df = pd.read_csv(master_data)
participant_IDs = pd.unique(df.loc[(df.exclude_from_analysis == False), 'participantID']).tolist()

for ID in participant_IDs:
    df.loc[df['participantID'] == ID, 'exclude_whole_row'] = False
df = df[(df['exclude_whole_row'] == False) & (df['match'] == 1.0)]
print(participant_IDs)


In [ ]:
for task_idx, task in enumerate(tasks):
    print(tasks[task_idx])

    for con in cons[task_idx]:
        print(con)
        master_cope_file_list = []
        master_varcope_file_list = []
        
        cope_file_list = []
        varcope_file_list = []
        group_cnts = []

        for group in groups:
            group_cnt = 0
    
            filesubs = pd.unique(df.loc[(df.exclude_whole_row == False) & (df.match == 1.0) &
                                        (df.func_task == task) & (df.group == group),'participantID']).tolist()
            for sub in filesubs:
                if iterative:
                    #iterative whole brain
                    con_img_fname = '{}/{}_{}_fold_{}_exclude_none_{}_cope.nii.gz'.format(copes_dir, sub, task, '*', con)
                    varcope_img_fname = '{}/{}_{}_fold_{}_exclude_none_{}_varcope.nii.gz'.format(varcopes_dir, sub, task, '*', con)

                    con_matches = glob.glob(con_img_fname)
                    varcope_matches = glob.glob(varcope_img_fname)
                    if len(con_matches) > 1 or len(varcope_matches) > 1:
                        print('There are duplicate files present.')
                    elif len(con_matches) < 1 or len(varcope_matches) < 1:
                        print(con_img_fname, 'is missing!')
                    else:
                        cope_file_list.append(con_matches[0])
                        varcope_file_list.append(varcope_matches[0])
                        group_cnt = group_cnt + 1
                else:
                    # non iterative whole brain
                    con_img_fname = '{}/{}_{}_{}_cope.nii.gz'.format(copes_dir, sub, task, con)
                    varcope_img_fname = '{}/{}_{}_{}_varcope.nii.gz'.format(varcopes_dir, sub, task, con)

                    if not op.isfile(con_img_fname):
                        print(con_img_fname, 'is missing!')
                    else:
                        cope_file_list.append(con_img_fname)
                        varcope_file_list.append(varcope_img_fname)
                        group_cnt = group_cnt + 1
            
            # actually do the merging, per group
            flame_outputdir = '{}/{}_{}_betweengroups'.format(flame_outputdir_root, task,con)
            os.makedirs(flame_outputdir, exist_ok = True)
            copefile = flame_outputdir + '/all_copes_{}.nii.gz'.format(group)
            cmd = 'fslmerge -t ' + copefile + ' ' 

            cmd = cmd + ' '.join(cope_file_list)
            result = subprocess.run(cmd, stdout=subprocess.PIPE, shell = True)
            print("Merged copes for {}".format(group), result.stdout)

            varcopefile = flame_outputdir + '/all_varcopes_{}.nii.gz'.format(group)
            cmd = 'fslmerge -t ' + varcopefile + ' ' 
            cmd = cmd + ' '.join(varcope_file_list)

            result = subprocess.run(cmd, stdout=subprocess.PIPE, shell = True)
            print("Merged varcopes for {}".format(group), result.stdout)
            
            group_cnts.append(group_cnt)
            
            master_cope_file_list += cope_file_list
            master_varcope_file_list += varcope_file_list
            cope_file_list = []
            varcope_file_list = []
        print(group_cnts)


        if do_allsubs_average:
            flame_whole_group_outputdir = '{}/{}_{}_allsubs'.format(flame_outputdir_root, task, con)
            os.makedirs(flame_whole_group_outputdir, exist_ok = True)
            
            copefile = flame_outputdir + '/all_copes.nii.gz'
            cmd = 'fslmerge -t ' + copefile + ' ' 
            cmd = cmd + ' '.join(master_cope_file_list)
            print(len(master_cope_file_list))
            result = subprocess.run(cmd, stdout=subprocess.PIPE, shell = True)
            print("Merged all copes", result.stdout)
            
            varcopefile = flame_outputdir + '/all_varcopes.nii.gz'
            cmd = 'fslmerge -t ' + varcopefile + ' ' 
            cmd = cmd + ' '.join(master_varcope_file_list)
            result = subprocess.run(cmd, stdout=subprocess.PIPE, shell = True)
            print("Merged all varcopes", result.stdout)
            
            shutil.copy2(copefile, flame_whole_group_outputdir)
            shutil.copy2(varcopefile, flame_whole_group_outputdir)
            
print('All done!')